In [1]:
import os
import torch 
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision import transforms
from torchvision.io import read_image



from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

In [2]:
## Data loader

def read_targets(tgt_path):
    
    file = open(tgt_path, 'r')
    lines = file.readlines()
    targets = {}
    targets['labels'] = []
    targets['boxes'] = []
#     print(len(lines))
    if len(lines) != 0:
        for line in lines:

            line = line.split('\n')[0]
            line = line.split(' ')
            line = [float(item) for item in line]
            if len(line) == 5:

                #Labels
                targets['labels'].append(line[0])
                #Boxes
                x, y, w, h = line[1:]
                targets['boxes'].append([x - 0.5*w, y - 0.5*h, x + 0.5*w, y + 0.5*h])
        targets['labels'] = torch.tensor(targets['labels'], dtype = torch.int64)
        targets['boxes'] = torch.tensor(targets['boxes'], dtype = torch.float)
    else:
        targets['labels'] = torch.tensor([0])
        targets['labels'].type(torch.int64)
        targets['boxes'] = torch.tensor([[float('nan')]*4], dtype = torch.float)
            

    return targets

def collate_fn(batch):
    return tuple(zip(*batch))
            

class ImageDataset(Dataset):
    def __init__(self, data_dir, transform=None, target_transform=None):
        img_dir = os.path.join(data_dir,'images')
        lbl_dir = os.path.join(data_dir,'labels')
        self.img_ID = [os.path.splitext(img)[0] for img in os.listdir(img_dir)]
        self.img_type = [os.path.splitext(img)[1] for img in os.listdir(img_dir)]
        self.img_dir = img_dir
        self.lbl_dir = lbl_dir
        self.transform = lambda x : -1 + 2*(x/torch.max(x))
        #self.transform = torchvision.transforms.Normalize()
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_ID)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, f'{self.img_ID[idx]}{self.img_type[idx]}')
        image = read_image(img_path, torchvision.io.ImageReadMode.UNCHANGED).to(torch.float32)
        
        lbl_path = os.path.join(self.lbl_dir, f'{self.img_ID[idx]}.txt')
        target = read_targets(lbl_path)
        
        image_ID = self.img_ID[idx]

        if self.transform:
            image = self.transform(image)
        
        return image, target, image_ID
    
class ImageDataModule():
    def __init__(self, data_dir, batch_size = 8, cuda = False):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.transform = transforms.Compose([transforms.ToTensor()])
        self.target_transform = transforms.Compose([transforms.ToTensor()])
        self.cuda = cuda

    def setup(self, stage: str):

        # Assign train/val datasets for use in dataloaders
        if stage == "fit":
            self.image_train = ImageDataset(self.data_dir + '/train/')
            self.image_val = ImageDataset(self.data_dir + '/valid/')

        if stage == "test":
            self.image_test = ImageDataset(self.data_dir + '/test/')
            
        if stage == "outliers":
            self.image_outliers = ImageDataset(self.data_dir + '/outliers/')
    def train_dataloader(self):
        
        if self.cuda:
            sampler = DistributedSampler(self.image_train)
        else:
            sampler = None
            
        return DataLoader(self.image_train,
                          batch_size=self.batch_size,
                          pin_memory=True,
                          shuffle=False,
                          sampler = sampler,
                          collate_fn=collate_fn)
         
    def valid_dataloader(self):
        
        
        if self.cuda:
            sampler = DistributedSampler(self.image_val)
        else:
            sampler = None
            
        return DataLoader(self.image_val,
                          batch_size=self.batch_size,
                          pin_memory=True,
                          shuffle=False,
                          sampler=sampler,
                          collate_fn=collate_fn)
    

    def test_dataloader(self):
        return DataLoader(self.image_test, batch_size=self.batch_size,
                          collate_fn=collate_fn)
    
    def outliers_dataloader(self):
        return DataLoader(self.image_outliers, batch_size=self.batch_size,
                          collate_fn=collate_fn)

class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0



In [3]:
data_dir = './data/mars_rover'
datamodule = ImageDataModule(data_dir)

datamodule.setup('fit')

train_data_loader = datamodule.train_dataloader()
for images, targets, imgID in train_data_loader:
    for target in targets:
        print(target['boxes'].shape)
valid_data_loader = datamodule.valid_dataloader()
for images, targets, imgID in valid_data_loader:
    for target in targets:
        print(target['boxes'].shape)

torch.Size([6, 4])
torch.Size([1, 4])
torch.Size([15, 4])
torch.Size([11, 4])
torch.Size([5, 4])
torch.Size([12, 4])
torch.Size([6, 4])
torch.Size([14, 4])
torch.Size([4, 4])
torch.Size([10, 4])
torch.Size([18, 4])
torch.Size([12, 4])
torch.Size([1, 4])
torch.Size([2, 4])
torch.Size([1, 4])
torch.Size([11, 4])
torch.Size([5, 4])
torch.Size([10, 4])
torch.Size([9, 4])
torch.Size([7, 4])
torch.Size([3, 4])
torch.Size([4, 4])
torch.Size([8, 4])
torch.Size([13, 4])
torch.Size([3, 4])
torch.Size([6, 4])
torch.Size([5, 4])
torch.Size([15, 4])
torch.Size([17, 4])
torch.Size([5, 4])
torch.Size([8, 4])
torch.Size([7, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([17, 4])
torch.Size([20, 4])
torch.Size([2, 4])
torch.Size([6, 4])
torch.Size([14, 4])
torch.Size([4, 4])
torch.Size([12, 4])
torch.Size([17, 4])
torch.Size([4, 4])
torch.Size([5, 4])
torch.Size([15, 4])
torch.Size([2, 4])
torch.Size([1, 4])
torch.Size([1, 4])
torch.Size([3, 4])
torch.Size([9, 4])
torch.Size([14, 4])
torch.Size([

torch.Size([9, 4])
torch.Size([11, 4])
torch.Size([6, 4])
torch.Size([3, 4])
torch.Size([6, 4])
torch.Size([31, 4])
torch.Size([9, 4])
torch.Size([6, 4])
torch.Size([3, 4])
torch.Size([9, 4])
torch.Size([7, 4])
torch.Size([4, 4])
torch.Size([27, 4])
torch.Size([17, 4])
torch.Size([4, 4])
torch.Size([18, 4])
torch.Size([13, 4])
torch.Size([10, 4])
torch.Size([18, 4])
torch.Size([46, 4])
torch.Size([17, 4])
torch.Size([17, 4])
torch.Size([20, 4])
torch.Size([10, 4])
torch.Size([3, 4])
torch.Size([11, 4])
torch.Size([21, 4])
torch.Size([14, 4])
torch.Size([6, 4])
torch.Size([16, 4])
torch.Size([28, 4])
torch.Size([31, 4])
torch.Size([15, 4])
torch.Size([1, 4])
torch.Size([13, 4])
torch.Size([24, 4])
torch.Size([5, 4])
torch.Size([3, 4])
torch.Size([22, 4])
torch.Size([27, 4])
torch.Size([7, 4])
torch.Size([14, 4])
torch.Size([25, 4])
torch.Size([9, 4])
torch.Size([31, 4])
torch.Size([6, 4])
torch.Size([19, 4])
torch.Size([3, 4])


In [4]:
## Model

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

in_features = model.roi_heads.box_predictor.cls_score.in_features
num_classes = 2

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)



/home/alcose/anaconda3/envs/rcnn/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/alcose/anaconda3/envs/rcnn/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
## TRAINER

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
lr_scheduler = None

num_epochs = 2


loss_hist = Averager()
itr = 1

for epoch in range(num_epochs):
    loss_hist.reset()
    
    for images, targets, imgID in train_data_loader:
        print(itr)
        images = list(image.to(device) for image in images)
        
#         for idx, target in enumerate(targets):
#             print(target)
#             print(imgID[idx])
        
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        try:
            loss_dict = model(images, targets)
        except:
            continue
        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()

        loss_hist.send(loss_value)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if itr % 50 == 0:
            print(f"Iteration #{itr} loss: {loss_value}")

        itr += 1
    
    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()

    print(f"Epoch #{epoch} loss: {loss_hist.value}")   

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
41
42
43
44
45
46
47
48
49
Epoch #0 loss: nan
50
Iteration #50 loss: nan
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
90
91
92
93
94
95
96
97
98
Epoch #1 loss: nan
